In [1]:
from hera.workflows import DAG, Workflow, script
from hera.shared import global_config as hera_config

hera_config.host = "http://localhost:2746"
hera_config.namespace = 'argo'
hera_config.verify_ssl = False

@script(image='python:3.12-slim')
def echo(message: str):
    print(message)


with Workflow(
    generate_name="dag-diamond-",
    entrypoint="diamond",
) as w:
    with DAG(name="diamond"):
        A = echo(name="A", arguments={"message": "A"})
        B = [
            echo(name="B0", arguments={"message": "B0"}),
            echo(name="B1", arguments={"message": "B1"})]
        C = echo(name="C", arguments={"message": "C"})

        A >> [B[0], B[1]] >> C
ret = w.create()

In [2]:
name = ret.metadata.name
name

'dag-diamond-m5d7f'

In [3]:
ret = w.wait()
ret.status.phase

'Succeeded'

In [4]:
from hera.workflows.service import WorkflowsService

service= WorkflowsService()

In [5]:
# For some reason, `service.workflow_logs()` doesn't work.  Here is an alternative:

import requests
import json

def logs(_name):
    url = f"{hera_config.host}/api/v1/workflows/{hera_config.namespace}/"\
          f"{_name}/log?logOptions.container=main"
    response = requests.get(url)

    for s in response.text.splitlines():
        obj = json.loads(s)['result']
        print(f"{obj['podName']}: {obj['content']}")

logs(name)

dag-diamond-m5d7f-echo-1738800557: A
dag-diamond-m5d7f-echo-1738800557: time="2024-10-13T01:59:59.835Z" level=info msg="sub-process exited" argo=true error="<nil>"
dag-diamond-m5d7f-echo-857775647: B1
dag-diamond-m5d7f-echo-840998028: B0
dag-diamond-m5d7f-echo-840998028: time="2024-10-13T02:00:09.875Z" level=info msg="sub-process exited" argo=true error="<nil>"
dag-diamond-m5d7f-echo-857775647: time="2024-10-13T02:00:09.876Z" level=info msg="sub-process exited" argo=true error="<nil>"
dag-diamond-m5d7f-echo-1705245319: C
dag-diamond-m5d7f-echo-1705245319: time="2024-10-13T02:00:19.891Z" level=info msg="sub-process exited" argo=true error="<nil>"


In [6]:
# service.delete_workflow(namespace=hera_config.namespace, name=name)